In [8]:
from ultralytics import YOLO
import os
import cv2
import shutil
from sklearn.model_selection import train_test_split
import numpy as np

In [9]:
import os
import shutil
import cv2
import numpy as np

# Укажите новые пути
IMAGES_TRAIN_DIR = r"C:\datadata\split_orig_data\open_img\train"
IMAGES_TEST_DIR = r"C:\datadata\split_orig_data\open_img\test"
MASKS_TRAIN_DIR = r"C:\datadata\split_orig_data\open_msk\train"
MASKS_TEST_DIR = r"C:\datadata\split_orig_data\open_msk\test"
OUTPUT_DIR = r'C:\datadata\split_orig_data\split_orig_aug_data_yolo'

# Создание структуры папок
os.makedirs(os.path.join(OUTPUT_DIR, 'images/train'), exist_ok=True)
os.makedirs(os.path.join(OUTPUT_DIR, 'images/val'), exist_ok=True)
os.makedirs(os.path.join(OUTPUT_DIR, 'labels/train'), exist_ok=True)
os.makedirs(os.path.join(OUTPUT_DIR, 'labels/val'), exist_ok=True)

def convert_mask_to_yolo(mask_path, output_subdir):
    """Конвертирует маску в формат YOLO."""
    image = cv2.imread(mask_path)
    if image is None:
        print(f"Не удалось открыть изображение: {mask_path}")
        return

    height, width = image.shape[:2]
    black_mask = cv2.inRange(image, (0, 0, 0), (50, 50, 50))
    new_image = np.ones_like(image) * 255
    new_image[black_mask > 0] = [0, 0, 0]
    gray_image = cv2.cvtColor(new_image, cv2.COLOR_BGR2GRAY)
    contours, _ = cv2.findContours(gray_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    output_file_name = os.path.splitext(os.path.basename(mask_path))[0] + '.txt'
    output_file_path = os.path.join(OUTPUT_DIR, output_subdir, output_file_name)

    with open(output_file_path, 'w') as f:
        for contour in contours:
            contour_points = contour.reshape(-1, 2)
            normalized_points = [(x / width, y / height) for x, y in contour_points]
            points_str = ' '.join(f"{x:.3f} {y:.3f}" for x, y in normalized_points)
            f.write(f"0 {points_str}\n")

def process_data(images_dir, masks_dir, output_images_subdir, output_labels_subdir):
    """Копирует изображения и конвертирует маски."""
    image_files = [f for f in os.listdir(images_dir) if f.endswith(('.jpg', '.png'))]
    for img in image_files:
        # Копирование изображения
        shutil.copy(os.path.join(images_dir, img), os.path.join(OUTPUT_DIR, output_images_subdir, img))
        # Конвертация маски
        mask_name = img.replace('.jpg', '.png')
        mask_path = os.path.join(masks_dir, mask_name)
        if os.path.exists(mask_path):
            convert_mask_to_yolo(mask_path, output_labels_subdir)
        else:
            print(f"Маска отсутствует для изображения: {img}")

# Обработка обучающего и тестового набора
process_data(IMAGES_TRAIN_DIR, MASKS_TRAIN_DIR, 'images/train', 'labels/train')
process_data(IMAGES_TEST_DIR, MASKS_TEST_DIR, 'images/val', 'labels/val')

# Создание файла data.yaml
data_yaml_content = f"""
train: {os.path.join(OUTPUT_DIR, 'images/train')}
val: {os.path.join(OUTPUT_DIR, 'images/val')}

nc: 1  # Обновите количество классов (1 для загрязнения)
names: ['contaminated']  # Обновите названия классов
"""

yaml_path = os.path.join(OUTPUT_DIR, 'data.yaml')
with open(yaml_path, 'w') as f:
    f.write(data_yaml_content)

print(f"Датасет успешно обработан и сохранен в: {OUTPUT_DIR}")

Датасет успешно обработан и сохранен в: C:\datadata\split_orig_data\split_orig_aug_data_yolo


In [10]:
model = YOLO("yolo11n-seg.pt")

In [ ]:
import torch


torch.set_num_threads(1)

train_results = model.train(
    data=r"C:\datadata\split_orig_data\split_orig_aug_data_yolo\data.yaml",  # path to dataset YAML
    epochs=50,  # number of training epochs
    imgsz=640,  # training image size
    device=0,  # device to run on, i.e. device=0 or device=0,1,2,3 or device=cpu
    patience=50,
    batch=16,
)

New https://pypi.org/project/ultralytics/8.3.47 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.38  Python-3.11.0 torch-2.0.1+cu117 CUDA:0 (NVIDIA GeForce RTX 4060, 8188MiB)
engine\trainer: task=segment, mode=train, model=yolo11n-seg.pt, data=C:\datadata\split_orig_data\split_orig_aug_data_yolo\data.yaml, epochs=100, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=0, workers=8, project=None, name=train4, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retin

100%|██████████| 5.35M/5.35M [00:01<00:00, 5.47MB/s]


AMP: checks passed 


train: Scanning C:\datadata\split_orig_data\split_orig_aug_data_yolo\labels\train... 314 images, 108 backgrounds, 0 corrupt: 100%|██████████| 314/314 [00:00<00:00, 420.68it/s]


train: New cache created: C:\datadata\split_orig_data\split_orig_aug_data_yolo\labels\train.cache


...

    100/100      3.79G     0.6328     0.9958     0.4146     0.8575         53        640: 100%|██████████| 20/20 [00:05<00:00,  3.82it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100%|██████████| 3/3 [00:00<00:00,  5.47it/s]

                   all         83         68      0.819        0.6      0.716      0.448      0.727      0.559      0.659      0.364



100 epochs completed in 0.201 hours.
Optimizer stripped from runs\segment\train4\weights\last.pt, 6.0MB
Optimizer stripped from runs\segment\train4\weights\best.pt, 6.0MB

Validating runs\segment\train4\weights\best.pt...
Ultralytics 8.3.38  Python-3.11.0 torch-2.0.1+cu117 CUDA:0 (NVIDIA GeForce RTX 4060, 8188MiB)
YOLO11n-seg summary (fused): 265 layers, 2,834,763 parameters, 0 gradients, 10.2 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100%|██████████| 3/3 [00:00<00:00,  3.73it/s]


                   all         83         68      0.751      0.706      0.772      0.477      0.763      0.574      0.668       0.36
Speed: 0.4ms preprocess, 2.5ms inference, 0.0ms loss, 1.0ms postprocess per image
Results saved to runs\segment\train4
